In [1]:
#Global imports
import cv2
import numpy as np
from os.path import isfile, join, basename, dirname
#For P1
from keras.models import load_model
#For P2
import tensorflow as tf
from scipy.stats import pearsonr
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

C:\Users\ctinipuclla\AppData\Local\miniconda3\envs\demo_env\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
model = load_model("model.hdf5")

C:\Users\ctinipuclla\AppData\Local\miniconda3\envs\demo_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ctinipuclla\AppData\Local\miniconda3\envs\demo_env\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [3]:
#functions for P1
def resize_frames_single(list_frames, target_size=(64,64)):
  resized_frames = [cv2.resize(frame, target_size, interpolation=cv2.INTER_AREA) for frame in list_frames]
  return resized_frames
def frame_difference_single(processed_frames):
  differences_video = []
  for j in range(len(processed_frames)-1):
    diff = cv2.absdiff(processed_frames[j], processed_frames[j+1])
    differences_video.append(diff)
  differences_video = np.array(differences_video)
  return differences_video
def generate_frame_batches_single(difference_frames, batch_size=36):
  batches = []

  n_frames = len(difference_frames)
  for start in range(0, n_frames, batch_size):
    end = start + batch_size
    # Ensure the batch has the required batch_size, otherwise fill with zeros
    if end <= n_frames:
      batch = difference_frames[start:end]
    else:
      # If there are not enough frames left for a full batch, fill the remainder with zeros
      remainder = end - n_frames
      batch = np.vstack((difference_frames[start:n_frames], np.zeros((remainder, difference_frames.shape[1], difference_frames.shape[2], difference_frames.shape[3]), dtype=difference_frames.dtype)))
    batches.append(batch)
  return batches

#Functions for P2
#Model for feature extraction
def build_cnn(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(16, (3, 3), activation='relu', strides=2))
    model.add(Conv2D(16, (3, 3), activation='relu', strides=2))
    model.add(MaxPooling2D((3, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(8192, activation='relu')) # andescon
    return model

#Extract features
def extract_features_batch(model, frames_batch):
    frames_reshaped = np.array(frames_batch).reshape((len(frames_batch), frames_batch[0].shape[0], frames_batch[0].shape[1], 1))
    features_batch = model.predict(frames_reshaped, verbose=0)
    return features_batch

#R: Pearson correlation coefficient f.
def calculate_r(features1, features2):
    r, _ = pearsonr(features1, features2)
    if r == None:
        return 0
    else:
        return r

#Rs: Difference of the adjacent correlation coefficients f.
def calculate_rs(r_video):
    rs_video = np.abs(np.diff(r_video, n=1))
    return np.insert(rs_video, 0, 0)

#Sod: Second-order derivative f.
def calculate_sod(differences):
    n = len(differences)
    second_deriv = np.zeros_like(differences)  # Initialize with zeros
    for x in range(1, n-1):
        second_deriv[x] = differences[x+1] + differences[x-1] - (2*differences[x])
    second_deriv[0] = 0  # Setting boundary conditions based on your specific needs
    if n > 1:
        second_deriv[-1] = 0
    return second_deriv #np.diff(differences, n=2)

# Video forgery detection f. -- andescon
def vf_detection(rs_video, lambda_t):
    return np.array(rs_video) > lambda_t

# VFD-video result -- andescon
def get_vfd_output(vfd_list):
    for i in range(vfd_list.size):
        if(vfd_list[i] == True):
            vfd_list[i] = 1 
        else:
            vfd_list[i] = 0
    return vfd_list

In [ ]:
#For interfaces
import ipywidgets as widgets
from IPython.display import display, Video
import os

#For preprocessing and models
import matplotlib.pyplot as plt

def preprocess_video(video_filename, method, frame_size=(250, 250)):
    cap = cv2.VideoCapture(video_filename)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_filename}")
        return None
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = []
    
    if method == 2:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # to grayscale
            resized_frame = cv2.resize(gray_frame, frame_size) # resize the frame
            frames.append(resized_frame)
        
        cap.release() # release the video capture object
        frames_array = np.array(frames)
    else:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        video_frames = np.array(frames)
        cap.release()
        #resize
        processed_frames = resize_frames_single(video_frames, frame_size)
        #difference
        difference_frames = frame_difference_single(processed_frames)
        #normalization
        difference_frames = difference_frames.astype('float32')/255
        #batches of 36 frames
        video_frame_batches = generate_frame_batches_single(difference_frames, batch_size = 36)
        video_frame_batches = np.array(video_frame_batches)
        frames_array = video_frame_batches
    
    if len(frames) == 0:
        print("Warning: No frames were extracted from the video.")
        return None
    
    return frames_array

# Crear widgets
upload_button = widgets.FileUpload(description="Cargar video")
output_video = widgets.Output()
run_p1_button = widgets.Button(description="Ejecutar P1")
run_p2_button = widgets.Button(description="Ejecutar P2")
result_output = widgets.Output()

# Función para mostrar el video cargado
def on_upload_change(change):
    with output_video:
        output_video.clear_output()
        # Elimina el archivo anterior si existe
        if hasattr(on_upload_change, 'last_video_filename'):
            if os.path.exists(on_upload_change.last_video_filename):
                try:
                    os.remove(on_upload_change.last_video_filename)
#                     print(f"Archivo anterior {on_upload_change.last_video_filename} eliminado.")
                except Exception as e:
                    print(f"Error al eliminar el archivo anterior: {str(e)}")

        for file_info in upload_button.value:
            video_data = file_info['content']
            video_filename = file_info['name']
            
#             print(f"Tamaño del archivo: {len(video_data)} bytes")
            
            with open(video_filename, 'wb') as f:
                f.write(video_data)
            
            if os.path.exists(video_filename):
                print(f"Video guardado y listo para procesar: '{video_filename}', con peso: {os.path.getsize(video_filename)/1000000} MB")
            else:
                print(f"Error: el archivo {video_filename} no fue guardado correctamente.")
                
            run_p2_action.video_filename = video_filename  # Guardar el nuevo nombre para run_p2_action
            run_p1_action.video_filename = video_filename  # Guardar el nuevo nombre para run_p1_action
            on_upload_change.last_video_filename = video_filename  # Guardar el nombre del último archivo

            try:
                display(Video(video_filename, width=600, height=400))
#                 print("Video mostrado correctamente.")
            except Exception as e:
                print(f"Error mostrando el video: {str(e)}")

upload_button.observe(on_upload_change, names='value')

# Funciones para ejecutar los modelos P1 y P2
def run_p1_action(b):
    with result_output:
        result_output.clear_output()
        # Aquí deberías llamar a tu función para ejecutar el modelo P1
        # Por ejemplo, podrías tener algo como:
        # result = model_P1(video_filename)
        if not hasattr(run_p1_action, 'video_filename'):
            print("Error: No video file available for processing.")
            return
        print(f"Ejecutando clasificación P1")
        video_filename = run_p1_action.video_filename
        video_frame_batches = preprocess_video(video_filename,1,(64,64))
        predictions = model.predict(video_frame_batches)
        predictions = predictions.reshape((-1))
        prediction = ""
        for i in range(predictions.shape[0]):
            if predictions[i]>0.5:
                predictions[i] = 1
            else:
                predictions[i] = 0
        if any(l==1 for l in predictions):
            forged_ind = np.where(predictions == 1)[0]
            prediction=f"El video es manipulado, en el segundo {forged_ind[0]/30} aproximadamente"
        else:
            prediction = "Video es original"
        print(prediction)
     
def run_p2_action(b):
    with result_output:
        result_output.clear_output()
        
        # Assuming the video filename was saved during the upload
        if not hasattr(run_p2_action, 'video_filename'):
            print("Error: No video file available for processing.")
            return        
        video_filename = run_p2_action.video_filename
        
        # Preprocess the video
        frames_array = preprocess_video(video_filename, 2)
        if frames_array is not None:
            print(f"Fueron procesados {frames_array.shape[0]} frames.")
            print(f"Ejecutando clasificación P2")
            #Execution
            r_videos = [] #PCC from video
            model = build_cnn(input_shape=(250, 250, 1)) #defining model
            
            features_batch = extract_features_batch(model, frames_array)
                    
            correlations = [calculate_r(features_batch[i], features_batch[i+1]) for i in range(len(frames_array)-1)]
#             r_videos.append(correlations)
            rs_video = calculate_rs(correlations)
#             print(f"Frame differences: {rs_video}")
            
            vfd_list = vf_detection(rs_video, 0.0085) #lambda value
#             print(f"VFD list: {vfd_list}")
            
            predictions = get_vfd_output(vfd_list)
            if any(l==1 for l in predictions):
                forged_ind = np.where(predictions == 1)[0]
                prediction=f"El video es manipulado, en el segundo {forged_ind[0]/30} aproximadamente"
            else:
                prediction = "Video es original"
        else:
            result = "Error processing the video."
        print(prediction)

frames_arraysito = run_p2_action
    
run_p1_button.on_click(run_p1_action)
run_p2_button.on_click(run_p2_action)

# Mostrar widgets en la interfaz
display(upload_button)
display(output_video)
display(run_p1_button, run_p2_button)
display(result_output)


FileUpload(value=(), description='Cargar video')

Output()

Button(description='Ejecutar P1', style=ButtonStyle())

Button(description='Ejecutar P2', style=ButtonStyle())

Output()